In [4]:
# fast calculate Geodistance in the earth 


import math

EARTH_RADUIS = 6378.137    # kilometre

def GeoDistance(lat1, long1, lat2, long2, decimal_digits=6):
    assert abs(lat1) <= 90 and abs(lat2) <= 90, "invalid latitude"
    assert abs(long1) <= 180 and abs(long2) <= 180, "invalid longtigude"
    rad_lat1 = math.radians(lat1)
    rad_lat2 = math.radians(lat2)
    rad_long1 = math.radians(long1)
    rad_long2 = math.radians(long2)
    a = rad_lat1 - rad_lat2
    b = rad_long1 - rad_long2
    s = (2 * math.asin(math.sqrt(math.pow(math.sin(a / 2), 2) +
         math.cos(rad_lat1) * math.cos(rad_lat2) *
         math.pow(math.sin(b / 2), 2))))
    distance = s * EARTH_RADUIS*1000   #transfer to m
    return round(distance,decimal_digits)

# print(GeoDistance(-35.308228, 149.124358, -33.868883, 151.209301))

In [ ]:
import numpy as np
import pandas as pd
import folium
# from geopy.distance import geodesic
df=pd.read_csv("Navy_yard.csv")
df = pd.DataFrame(df, columns=['name','X','Y'])
df = df.dropna()
df.head()
m = folium.Map(location=[39.895243, -75.188212], zoom_start=14)

latitudes = list(df.Y)
longitudes = list(df.X)
labels = list(df.name)

# openstreetmap

for lat, lng, label in zip(latitudes, longitudes,labels):
    folium.Marker([lat, lng], popup=label).add_to(m)    
# m
m.save('Navy_yard_building_label.html')

from folium.plugins import HeatMap

# let's start again with a clean copy of the map of San Francisco
m = folium.Map(location = [ 39.8951467, -75.1896499,

                          ], zoom_start = 15)

# Convert data format
heatdata = df[['Y','X']].values.tolist()

# add incidents to map
HeatMap(heatdata).add_to(m)

m.save('Navy_yead_heatmap.html')

In [ ]:
df = pd.DataFrame(df, columns=['name','No.','X','Y'])
for i in range(len(df)):
    df.iloc[i,1] = int(i)
df.to_csv("sheet2.csv")

x = df.values.tolist()
sheet3 = []
for i in x:
    a = int(i[1])
    lng1 = i[3]
    lat1 = i[2]
#     print(a)
#     print(a,lng1,lat1)
    for j in x[a+1:]:
        b = a + 1
        lng2 = j[3]
        lat2 = j[2]
        distance = GeoDistance((lng1,lat1), (lng2,lat2)).m 
        line = [a, b, distance]
        sheet3.append(line)
# print(sheet3)


sheet_3 = pd.DataFrame(sheet3, columns = ['A','B','Distance'])
sheet_3.head()

sheet_3['connection'] = sheet_3.Distance.apply(lambda x: 'strong' if x<800 else 'weak')
sheet_3

sheet_3.to_csv("sheet3.csv")

In [ ]:
# transfer GeoJson to CSV

# https://mygeodata.cloud/converter/geojson-to-csv
import json
import csv
import argparse
import sys
import os


def main(args):
    fp = args['infile']
    geojson_data = json.load(fp)
    if geojson_data['type'] == 'FeatureCollection':
        parse_feature_collection(geojson_data['features'], args['outfile'])
    else:
        print("Can currently only parse FeatureCollections, but I found ", geojson_data['type'], " instead")


def parse_feature_collection(features, outfile):
    # Each feature from the feature collection is a Type: Feature, a bunch of properties, and geometry.
    # We want to flatten those out

    # create the csv writer object
    csvwriter = csv.writer(outfile, lineterminator=os.linesep)

    count = 0
    # We'd like to save the first header we see, to maintain the exact same ordering, in case
    # some features change their order (we can't rely on it!)
    header = []
    for feature in features:
        if count == 0:
            header = list(feature['properties'].keys())
            # We're going to assume the feature is just a point for this stage
            header.extend(['px','py'])
            csvwriter.writerow(header)
            count += 1
        csvwriter.writerow(feature_to_row(feature, feature['properties'].keys()))
    outfile.close()

def feature_to_row(feature, header):
    l = []
    for k in header:
        l.append(feature['properties'][k])
    if feature['geometry']['type'] != 'Point':
        raise RuntimeError("Expecting point type, but got ", feature['geometry']['type'])
    coords = feature['geometry']['coordinates']
    assert(len(coords)==2)
    l.extend(coords)
    return l


if __name__ == "__main__":
    parser = argparse.ArgumentParser(prog="geojson2csv.py",
                                     description='Convert simple GeoJSONs to CSVs')
    parser.add_argument('infile', nargs='?', type=argparse.FileType('r'), default = sys.stdin)
    parser.add_argument('outfile', nargs='?', type=argparse.FileType('w'), default = sys.stdout)
    pargs = parser.parse_args()
    main(vars(pargs))


In [ ]:
# when you get the list of polygon, use this algorithm to obtain the central point of building

def get_centerpoint(lis):
    area = 0.0
    x,y = 0.0,0.0

    a = len(lis)
    for i in range(a):
        lat = lis[i][0] #weidu
        lng = lis[i][1] #jingdu

        if i == 0:
            lat1 = lis[-1][0]
            lng1 = lis[-1][1]

        else:
            lat1 = lis[i-1][0]
            lng1 = lis[i-1][1]

        fg = (lat*lng1 - lng*lat1)/2.0

        area += fg
        x += fg*(lat+lat1)/3.0
        y += fg*(lng+lng1)/3.0

    x = x/area
    y = y/area

    return x,y